<div style="text-align: right"><b> Ana Valentina López Chacón & Camila Palomeque </b></div>
<div style="text-align: right"><b> Herramientas y Aplicaciones de la Inteligencia Artificial, 2024 - 2025 </b></div>

## **Proyecto: Detección de Argumentos**

In [1]:
!pip install langchain pymupdf openai --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import List
from pydantic import BaseModel, Field, ValidationError
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
import requests
import json
import re
from openai import OpenAI
import openai
import httpx

In [3]:
import fitz 

def extract_text_from_pdf(pdf_path, page_list):
    doc = fitz.open(pdf_path)
    text = ""
    for page in page_list:
        if 0 <= page < len(doc):
            text += doc[page].get_text()
    return text

In [7]:
client = OpenAI(
    base_url = 'https://api.poligpt.upv.es',  
    api_key = 'sk-Icbf-5FyeV0QcLWBC9SNEA'     
)

timeout = httpx.Timeout(60.0, connect=30.0) 

In [15]:
class ArgumentResponse(BaseModel):
    arguments: list[str] = Field(..., description="List of arguments extracted directly from the text.")

pydantic_parser = PydanticOutputParser(pydantic_object=ArgumentResponse)

def extract_arguments_json(text: str, topic: str) -> ArgumentResponse:
    format_instructions = pydantic_parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            "You are an argument mining assistant.\n\n"
            "Extract the arguments directly from the text below, considering the following topic: {topic}.\n\n"
            "Text:\n\"\"\"\n{text}\n\"\"\"\n"
            "Respond ONLY with a JSON object like this:\n\n"
            "{format_instructions}\n\n"
            "Do not include markdown (```), explanations, or any extra text. Just output the raw JSON object."
        ),
        input_variables=["text", "topic"],
        partial_variables={"format_instructions": format_instructions},
    )

    final_prompt = prompt.format_prompt(text=text, topic=topic).to_string()

    chat_completion = client.chat.completions.create(
        messages = [
            {'role': 'system', 'content': 'You are an argument mining assistant.'},
            {'role': 'user', 'content': final_prompt}
        ],
        model = "poligpt",
        temperature = 0,
        # timeout = timeout
    )

    raw_output = chat_completion.choices[0].message.content

    print("Model Output:", raw_output)

    return pydantic_parser.parse(raw_output)

In [16]:
pdf_path = './SDGoals_UNESCO_en_2024.pdf'
text = extract_text_from_pdf(pdf_path, list(range(9, 10)))
print(text)

The Sustainable Development Goals Report 2024
8
No poverty
•	
The COVID-19 pandemic and subsequent shocks from 2020 to 2022 
have hampered global efforts to eradicate extreme poverty. The global 
extreme poverty rate increased in 2020 for the first time in decades, 
setting back progress by three years. Since then, recovery has been 
uneven, with low-income countries lagging behind. By 2030, 590 million 
people may still live in extreme poverty if current trends persist. Without 
a substantial acceleration in poverty reduction, fewer than 3 in 10 
countries are expected to halve national poverty by 2030.
•	
	Despite increasing efforts and commitments to expand social protection 
programmes, significant coverage gaps left 1.4 billion children uncovered 
in 2023. 
•	
	Climate change is hindering poverty reduction, and disasters result in 
millions of households becoming poor or remaining trapped in poverty. 
•	
Ending poverty requires a wide-ranging approach that combines 
comprehensive 

In [17]:
topic_g1 = "End poverty in all its forms everywhere"
extracted_arguments = extract_arguments_json(text, topic_g1)
print(extracted_arguments)

Model Output: {"arguments": [
"The COVID-19 pandemic and subsequent shocks from 2020 to 2022 have hampered global efforts to eradicate extreme poverty.",
"Climate change is hindering poverty reduction, and disasters result in millions of households becoming poor or remaining trapped in poverty.",
"Ending poverty requires a wide-ranging approach that combines comprehensive social protection systems, inclusive economic policies, investments in human capital, measures to address inequality and climate resilience, and international cooperation and partnership.",
"Without a substantial acceleration in poverty reduction, fewer than 3 in 10 countries are expected to halve national poverty by 2030.",
"By 2030, 590 million people may still live in extreme poverty if current trends persist.",
"Significant coverage gaps left 1.4 billion children uncovered in 2023 despite increasing efforts and commitments to expand social protection programmes."
]}
arguments=['The COVID-19 pandemic and subsequent

In [11]:
data_dict = extracted_arguments.model_dump()

with open("./all_args.json", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=2, ensure_ascii=False)